In [1]:
import sys
from time import sleep
from time import gmtime, strftime
import mmap
import resource
import numpy as np
import os
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import FileLink, FileLinks
from IPython.display import clear_output
import datetime
%matplotlib inline

# This sets up the FPGA and maps some memory locations to variables we can access
# Don't change anything here unless you enjoy things not working.
os.system('cat /root/system_wrapper.bit > /dev/xdevcfg')
#os.system('cat /opt/redpitaya/fpga/pet_rebuild.bit > /dev/xdevcfg')

fd = os.open('/dev/mem', os.O_RDWR)
mm = []
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41210000)) #0
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41300000)) #1
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41200000)) #2
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41230000)) #3
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41240000)) #4
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41250000)) #5
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41220000)) #6
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41260000)) #7
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41270000)) #8
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41280000)) #9
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41290000)) #10
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x412A0000)) #11
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x412B0000)) #12
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x412C0000)) #13
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x412D0000)) #14
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x42220000)) #15
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x412E0000)) #16
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x412F0000)) #17
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41310000)) #18
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41320000)) #19
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41330000)) #20
mm.append(mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x41340000)) #21

lat_in  = np.frombuffer(mm[0], dtype='uint32') 
lat_out = np.frombuffer(mm[1], dtype='uint32') 
rot_in  = np.frombuffer(mm[2], dtype='uint32') 
ch1_threshold = np.frombuffer(mm[3], dtype='uint32') 
ch2_threshold = np.frombuffer(mm[4], dtype='uint32') 
coincidence_settings = np.frombuffer(mm[5], dtype='uint16') 
threshold_invert_settings = np.frombuffer(mm[6], dtype='uint32') 
coincidence_timeout_counts  = np.frombuffer(mm[7], dtype='uint32') 
counts  =  np.frombuffer(mm[8], dtype='uint32') 
uthreshold_counts =  np.frombuffer(mm[9], dtype='uint32') 
coincidence_status =  np.frombuffer(mm[10], dtype='uint32') 
raw_input = np.frombuffer(mm[11],count=2,dtype='int16') # ADC data is stored as a signed integer
#sensor_readout = np.frombuffer(mm[11],dtype='uint32')
sensor_readout = np.frombuffer(mm[11],offset=8,count=1,dtype='uint32') # This skips the first 4 bytes, getting to the readout info.
threshold_detection = np.frombuffer(mm[12],dtype='uint32')
adc_module_signal = np.frombuffer(mm[13],dtype='uint32')
direct_adc = np.frombuffer(mm[14],dtype='uint32')
offset_settings = np.frombuffer(mm[15],dtype='uint32')
dac_settings = np.frombuffer(mm[16],dtype='uint32')
rotation_settings = np.frombuffer(mm[17],dtype='int32')

PHA_readout = np.frombuffer(mm[18],dtype='uint32')
PHA_stored_pulses = np.frombuffer(mm[19],dtype='uint32')
toggler  = np.frombuffer(mm[20],dtype='uint32')
scan_runtime = np.frombuffer(mm[21],dtype='uint32')


In [2]:
threshold_invert_settings[0] = 1
ch1_threshold[0] = 10000
ch1_threshold[2] = 12000
#sleep(7)
#print(PHA_stored_pulses[0])
print(PHA_readout[2])

1


In [3]:
'''
pulses = []
last = 0
current = 0
for i in range(10):
    while(PHA_stored_pulses[0]>0):
        toggler[0] = 1 
        current = PHA_readout[0]
        toggler[0] = 0   
        if (current != last and current != 0):
            pulses.append(current)
            last = current
            #print(current)
        if(len(pulses)>100000):
            break
    #sleep(1)
    print(len(pulses))
    
try:
    fig, ax = plt.subplots()
    ax.plot(np.sort(pulses)/max(pulses))
    fig2, ax2 = plt.subplots()
    toplot = np.sort(pulses)[:-len(pulses)//10]
    ax2.hist(toplot,200,log=True)
except:
    pass
print(PHA_readout[2])
print(len(pulses))
'''

'\npulses = []\nlast = 0\ncurrent = 0\nfor i in range(10):\n    while(PHA_stored_pulses[0]>0):\n        toggler[0] = 1 \n        current = PHA_readout[0]\n        toggler[0] = 0   \n        if (current != last and current != 0):\n            pulses.append(current)\n            last = current\n            #print(current)\n        if(len(pulses)>100000):\n            break\n    #sleep(1)\n    print(len(pulses))\n    \ntry:\n    fig, ax = plt.subplots()\n    ax.plot(np.sort(pulses)/max(pulses))\n    fig2, ax2 = plt.subplots()\n    toplot = np.sort(pulses)[:-len(pulses)//10]\n    ax2.hist(toplot,200,log=True)\nexcept:\n    pass\nprint(PHA_readout[2])\nprint(len(pulses))\n'

In [4]:
'''
print(format(toggler[2],"0>8b"))
print(PHA_readout[0])
toggler[0] = 1
print(format(toggler[2],"0>8b"))
print(PHA_readout[0])
toggler[0] = 0
print(format(toggler[2],"0>8b"))
print(PHA_readout[0])
toggler[0] = 1
print(format(toggler[2],"0>8b"))
print(PHA_readout[0])
'''

'\nprint(format(toggler[2],"0>8b"))\nprint(PHA_readout[0])\ntoggler[0] = 1\nprint(format(toggler[2],"0>8b"))\nprint(PHA_readout[0])\ntoggler[0] = 0\nprint(format(toggler[2],"0>8b"))\nprint(PHA_readout[0])\ntoggler[0] = 1\nprint(format(toggler[2],"0>8b"))\nprint(PHA_readout[0])\n'

In [5]:
#ch1_threshold[0] = 7000
#ch1_threshold[2] = 5000

ch1_threshold[0] = 8500
ch1_threshold[2] = 10000

ch2_threshold[0] = 8400
ch2_threshold[2] = 10000

offset_settings[0] = 0
offset_settings[2] = 0

print(offset_settings[0],offset_settings[2])
print(ch1_threshold[0],ch1_threshold[2])
threshold_invert_settings[0] = 1
threshold_invert_settings[2] = 1

#print(threshold_invert_settings[0])
print(format(threshold_detection[0],'#04b'))

print(direct_adc[0])
dac_settings[0]=4

0 0
8500 10000
0b11001100
8379


In [6]:
import threading
import time
class output_readout():
    def __init__(self,*args,**kwargs):
        print(args,kwargs)
        self.name = args[0]
        self.variable = args[1]
        self.widget = widgets.Label(value=self.name +": "+ str(eval(self.variable)))
        self.running = True
        self.update_rate = .1
        self.thread = threading.Thread(target=self.update_thread)
        self.thread.start()
        for key,val in kwargs.items():
            setattr(self,key,val)
        #if args[2] is not None:
        #    self.type = args[2]
    def __exit__(self):
        self.thread.stop()
    def get(self):
        return eval(self.variable)
    def update(self):
       # if self.type == "standard":
        if "single_bit" in vars(self):
            self.widget.value = self.name +": "+ str(  eval(self.variable) & 2**self.single_bit != 0)
        elif "dictionary" in vars(self):
            if eval(self.variable) in self.dictionary:
                self.widget.value = self.name +": "+ str(  self.dictionary[eval(self.variable)])
            else:
                self.widget.value = "Unknown state: " + str(eval(self.variable))
        elif "conversion" in vars(self):
            self.widget.value = self.name +": {:.3f} V ".format(eval(self.variable)/self.conversion)
        else:
            self.widget.value = self.name +": "+ str(eval(self.variable))
       # elif self.type == "ADC":
       #     self.widget.value = self.name +": "+ str(eval(self.variable)/2**10)
    def update_thread(self):
        while True:
            if self.running:
                self.update()
            time.sleep(self.update_rate)
    def stop(self):
        self.running=False
    def start(self):
        self.running=True

In [7]:
class input_readout():
    def __init__(self,*args,**kwargs):
        print(args,kwargs)
        self.name = args[0]
        self.variable = args[1]
        for key,val in kwargs.items():
            setattr(self,key,val)
        self.widget = widgets.Label(value=self.name +": "+ str(eval(self.variable)))
        if "single_bit" in vars(self):
            if "invert" in vars(self):
                self.set_widget = widgets.Checkbox(value=self.invert,description = "Set " + (self.name), indent=False)
            else:
                self.set_widget = widgets.Checkbox(value=False,description = "Set " + (self.name), indent=False)
        elif "dictionary" in vars(self):
            self.set_widget = widgets.Dropdown(options = list(zip(list(self.dictionary.values()),list(self.dictionary))),description = "Set " + (self.name))
        else:
            self.set_widget = widgets.IntText(description = "Set",value = eval(self.variable),step=10)
        self.update_value=True
        self.running = True
        if "update_rate" not in vars(self):
            self.update_rate = .5
        self.thread = threading.Thread(target=self.update_thread)
        self.thread.start()

        #if args[2] is not None:
        #    self.type = args[2]
    def __exit__(self):
        self.thread.stop()
    def get(self):
        return eval(self.variable)
    def update(self):
        if(self.update_value):
            if "single_bit" in vars(self):
                if self.set_widget.value:
                    self.widget.value = str(eval(self.variable) | (1<<self.single_bit))
                    exec(self.variable +"="+ str(eval(self.variable) | (1<<self.single_bit)))
                else:
                    self.widget.value = str(eval(self.variable) & ~(1<<self.single_bit))
                    exec(self.variable +"="+ str(eval(self.variable) & ~(1<<self.single_bit)))
            elif "conversion" in vars(self):
                self.widget.value = self.name +": {:.3f} V ".format(eval(self.variable)/self.conversion)
                exec(self.variable +"="+ str(self.set_widget.value))
            elif "dictionary" in vars(self):
                 self.widget.value = self.name +": "+ str(eval(self.variable))
                 exec(self.variable +"="+ str(self.set_widget.value))
            else:
                self.widget.value = self.name +": "+ str(eval(self.variable))
                exec(self.variable +"="+ str(self.set_widget.value))
    def update_thread(self):
        while True:
            if self.running:
                self.update()
            time.sleep(self.update_rate)
    def stop(self):
        self.running=False
    def start(self):
        self.running=True

In [8]:
class divider():
    def __init__(self):
        self.widget= widgets.HTML("<hr>")

In [9]:
coincidence_dict = {0:"Idle",
                    1:"Signal on ch 1",
                    2:"Signal on ch 2",
                    3:"Coincidence detected",
                    4:"Timeout no signal on 2",
                    5:"Coincidence detection loop finished",
                    6:"Error",
                    7:"Ch1 over threshold",
                    8:"Ch2 over threshold",
                    9:"Timeout no signal on 1",
                    10:"Recover"}
offset_settings[0] = 0
offset_settings[2] = 0
rotation_dict =    {0:"Idle",
                    1:"Running",
                    2:"Run Finished",
                    3:"Resetting",
                    4:"Timeout before reset finished",
                    19:"Resetting: First pass detected",
                    35:"Resetting: Second pass",
                    36:"Reset Timeout"}
lateral_dict =     {0:"Idle",
                    1:"Running",
                    2:"Run Finished",
                    3:"Resetting",
                    4:"Error",
                    19:"Resetting: First pass detected",
                    35:"Resetting: Second pass",
                    36:"Reset Timeout"}
DAC_dict =          {0:"Ch 1 Low Threshold",
                     1:"Ch 1 High Threshold",
                     2:"Ch 1 Input",
                     3:"CH 2 Input",
                     4:"Lateral Motor Signals",
                     5:"Rotational Motor Signals",
                     6:"Ch 2 Low Threshold",
                     7:"Ch 2 High Threshold",
                     8:"Ch 1 Low Pulse",
                     9:"Ch 2 Low Pulse",
                     128: "Ch 1 Both Thresholds",
                     129: "Ch 2 Both Thresholds"}

In [10]:
if "fpga_readouts" in locals():
    try:
        for item in children:
            item.close()
        fpga_readouts.clear()
    except:
        pass
fpga_readouts = []

hrule = divider()



fpga_readouts.append(output_readout("Lateral Max Sensor","sensor_readout[0]",single_bit=0))
fpga_readouts.append(output_readout("Lateral Zero Sensor","sensor_readout[0]",single_bit=1))
fpga_readouts.append(output_readout("Lateral Input Signals","lat_in[0]"))
fpga_readouts.append(input_readout("Lateral Start","lat_in[0]",single_bit=0))
fpga_readouts.append(input_readout("Lateral Direction","lat_in[0]",single_bit=1))
fpga_readouts.append(input_readout("Lateral Reset","lat_in[0]",single_bit=2))
fpga_readouts.append(input_readout("Lateral Verify","lat_in[0]",single_bit=3))
fpga_readouts.append(input_readout("Lateral Run Length","lat_in[2]"))
fpga_readouts.append(output_readout("Lateral Current State","lat_out[0]",dictionary = lateral_dict))
fpga_readouts.append(output_readout("Lateral Position","lat_out[2]"))
fpga_readouts.append(hrule)
fpga_readouts.append(input_readout("Rotational Input Signals","rot_in[0]"))
fpga_readouts.append(output_readout("Rotation Zero Sensor","sensor_readout[0]",single_bit=2))
fpga_readouts.append(input_readout("Rotational Start","rot_in[0]",single_bit=0))
fpga_readouts.append(input_readout("Rotational Direction","rot_in[0]",single_bit=1))
fpga_readouts.append(input_readout("Rotational Reset","rot_in[0]",single_bit=2))
fpga_readouts.append(input_readout("Rotational Verify","rot_in[0]",single_bit=3))
fpga_readouts.append(input_readout("Rotational Run Length Signals","rot_in[2]"))
fpga_readouts.append(output_readout("Rotational Current State","rotation_settings[0]",dictionary = rotation_dict))
fpga_readouts.append(output_readout("Rotational Position","rotation_settings[2]"))
fpga_readouts.append(hrule)
fpga_readouts.append(output_readout("Sensors","sensor_readout[0]"))
fpga_readouts.append(hrule)
fpga_readouts.append(output_readout("Ch1 Counts","counts[0]"))

fpga_readouts.append(input_readout("Ch1 Lower Threshold","ch1_threshold[0]",conversion = 2**14))
fpga_readouts.append(input_readout("Ch1 Upper Threshold","ch1_threshold[2]",conversion = 2**14))
fpga_readouts.append(input_readout("Ch1 Threshold Invert","threshold_invert_settings[0]",single_bit=0,invert=True))
fpga_readouts.append(output_readout("Ch1 Reading","direct_adc[0]"))
fpga_readouts.append(output_readout("Ch1 L Detected","threshold_detection[0]",single_bit=0))
fpga_readouts.append(output_readout("Ch1 H Detected","threshold_detection[0]",single_bit=1))
fpga_readouts.append(output_readout("Ch1 Sign","threshold_detection[0]",single_bit=2))
fpga_readouts.append(output_readout("Ch1 Invert Setting","threshold_detection[0]",single_bit=3))
#fpga_readouts.append(output_readout("Ch1 Reading","adc_module_signal[0]",conversion = 2**14))


fpga_readouts.append(hrule)
fpga_readouts.append(output_readout("Ch2 Counts","counts[2]"))

fpga_readouts.append(input_readout("Ch2 Lower Threshold","ch2_threshold[0]",conversion = 2**14))
fpga_readouts.append(input_readout("Ch2 Upper Threhold", "ch2_threshold[2]",conversion = 2**14))
fpga_readouts.append(input_readout("Ch2 Threshold Invert","threshold_invert_settings[2]",single_bit=0,invert=True))
fpga_readouts.append(output_readout("Ch2 Reading","direct_adc[2]"))
fpga_readouts.append(output_readout("Ch2 L Detected","threshold_detection[0]",single_bit=4))
fpga_readouts.append(output_readout("Ch2 H Detected","threshold_detection[0]",single_bit=5))
fpga_readouts.append(output_readout("Ch2 Sign","threshold_detection[0]",single_bit=6))
fpga_readouts.append(output_readout("Ch2 Invert Setting","threshold_detection[0]",single_bit=7))

fpga_readouts.append(hrule)
fpga_readouts.append(output_readout("Coincidences","coincidence_timeout_counts[0]"))
fpga_readouts.append(output_readout("Timeouts","coincidence_timeout_counts[2]"))
fpga_readouts.append(output_readout("Ch1 Overthreshold Counts","uthreshold_counts[0]"))
fpga_readouts.append(output_readout("Ch2 Overthreshold Counts","uthreshold_counts[2]"))
fpga_readouts.append(output_readout("Coincidence Status","coincidence_status[0]",dictionary = coincidence_dict))
fpga_readouts.append(output_readout("Coincidence Timer","coincidence_status[2]",update_rate = 0.01))
fpga_readouts.append(output_readout("Coincidence Settings","coincidence_settings[0]"))
fpga_readouts.append(input_readout("Coincidence Reset","coincidence_settings[0]",single_bit=0))
fpga_readouts.append(input_readout("Coincidence Run","coincidence_settings[0]",single_bit=1))
fpga_readouts.append(hrule)
fpga_readouts.append(input_readout("Out 1 Select","dac_settings[2]",dictionary = DAC_dict))
fpga_readouts.append(input_readout("Out 2 Select","dac_settings[0]",dictionary = DAC_dict))


children = []
for item in fpga_readouts:
    if "set_widget" in vars(item):
        children.append(widgets.HBox((item.widget,item.set_widget)))           
    else:
        children.append(item.widget)
data_pane = widgets.VBox(children)
display(data_pane)

('Lateral Max Sensor', 'sensor_readout[0]') {'single_bit': 0}
('Lateral Zero Sensor', 'sensor_readout[0]') {'single_bit': 1}
('Lateral Input Signals', 'lat_in[0]') {}
('Lateral Start', 'lat_in[0]') {'single_bit': 0}
('Lateral Direction', 'lat_in[0]') {'single_bit': 1}
('Lateral Reset', 'lat_in[0]') {'single_bit': 2}
('Lateral Verify', 'lat_in[0]') {'single_bit': 3}
('Lateral Run Length', 'lat_in[2]') {}
('Lateral Current State', 'lat_out[0]') {'dictionary': {0: 'Idle', 1: 'Running', 2: 'Run Finished', 3: 'Resetting', 4: 'Error', 19: 'Resetting: First pass detected', 36: 'Reset Timeout', 35: 'Resetting: Second pass'}}
('Lateral Position', 'lat_out[2]') {}
('Rotational Input Signals', 'rot_in[0]') {}
('Rotation Zero Sensor', 'sensor_readout[0]') {'single_bit': 2}
('Rotational Start', 'rot_in[0]') {'single_bit': 0}
('Rotational Direction', 'rot_in[0]') {'single_bit': 1}
('Rotational Reset', 'rot_in[0]') {'single_bit': 2}
('Rotational Verify', 'rot_in[0]') {'single_bit': 3}
('Rotational Ru

A Jupyter Widget

----